# Assigment 4


> Group:12 YanpingTang, Guangyu Ma, Weiyou Wang



In [3]:
import pandas as pd
import numpy as np
import torch

# Task 1

**For Colab users:** the following command downloads the first CSV file.

In [4]:
!wget --no-check-certificate https://www.cse.chalmers.se/~richajo/dit866/assignments/a4/data/a4_synthetic.csv

--2025-02-28 11:06:22--  https://www.cse.chalmers.se/~richajo/dit866/assignments/a4/data/a4_synthetic.csv
Resolving www.cse.chalmers.se (www.cse.chalmers.se)... 129.16.222.93
Connecting to www.cse.chalmers.se (www.cse.chalmers.se)|129.16.222.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5874 (5.7K) [text/csv]
Saving to: ‘a4_synthetic.csv’

a4_synthetic.csv    100%[===================>]   5.74K  --.-KB/s    in 0s      

2025-02-28 11:06:23 (1.61 GB/s) - ‘a4_synthetic.csv’ saved [5874/5874]



Loading the synthetic dataset.

In [9]:
# You may need to edit the path, depending on where you put the files.
data = pd.read_csv('a4_synthetic.csv')

X = data.drop(columns='y').to_numpy()
Y = data.y.to_numpy()

Training a linear regression model for this synthetic dataset.

In [10]:
np.random.seed(1)

w_init = np.random.normal(size=(2, 1))
b_init = np.random.normal(size=(1, 1))

# We just declare the parameter tensors. Do not use nn.Linear.
w = torch.tensor(w_init, dtype=torch.float32, requires_grad=True)# TODO: a tensor initialized as w_init
b = torch.tensor(b_init, dtype=torch.float32, requires_grad=True)# TODO: a tensor initialized as b_init

eta = 1e-2
opt = opt = torch.optim.SGD([w, b], lr=eta)# TODO: a SGD optimizer with a learning rate of eta

for i in range(10):

    sum_err = 0

    for row in range(X.shape[0]):
        x = torch.tensor(X[[row], :]).float()
        y = torch.tensor(Y[[row]]).float()

        # Forward pass.
        y_pred = x @ w + b# TODO: compute predicted value for x
        err = (y_pred - y) ** 2   # TODO: compute squared error loss

        # Backward and update.
        # TODO: compute gradients and then update the model.
        opt.zero_grad()
        err.backward()
        opt.step()
        # For statistics.
        sum_err += err.item()

    mse = sum_err / X.shape[0]
    print(f'Epoch {i+1}: MSE =', mse)

Epoch 1: MSE = 0.7999662599533621
Epoch 2: MSE = 0.017392394375863204
Epoch 3: MSE = 0.009377417723923998
Epoch 4: MSE = 0.009355327363746255
Epoch 5: MSE = 0.00936544022662929
Epoch 6: MSE = 0.009366988615365699
Epoch 7: MSE = 0.009367207542284177
Epoch 8: MSE = 0.009367238225554502
Epoch 9: MSE = 0.00936724472411015
Epoch 10: MSE = 0.009367244635918724


# Task 2

In [11]:
class Tensor:

    # Constructor. Just store the input values.
    def __init__(self, data, requires_grad=False, grad_fn=None):
        self.data = data
        self.shape = data.shape
        self.grad_fn = grad_fn
        self.requires_grad = requires_grad
        self.grad = None

    # So that we can print the object or show it in a notebook cell.
    def __repr__(self):
        dstr = repr(self.data)
        if self.requires_grad:
            gstr = ', requires_grad=True'
        elif self.grad_fn is not None:
            gstr = f', grad_fn={self.grad_fn}'
        else:
            gstr = ''
        return f'Tensor({dstr}{gstr})'

    # Extract one numerical value from this tensor.
    def item(self):
        return self.data.item()

    # YOUR WORK WILL BE DONE BELOW

    # For Task 2:

    # Operator +
    def __add__(self, right):
        # Call the helper function defined below.
        return addition(self, right)

    # Operator -
    def __sub__(self, right):
        newData = self.data-right.data
        grad_fn = SubNode(self, right)
        return Tensor(newData,grad_fn=grad_fn,requires_grad=True)

    # Operator @
    def __matmul__(self, right):
        newData = self.data@right.data
        grad_fn = MatMulNode(self, right)
        return Tensor(newData,grad_fn=grad_fn,requires_grad=True)

    # Operator **
    def __pow__(self, right):
        # NOTE! We are assuming that right is an integer here, not a Tensor!
        if not isinstance(right, int):
            raise Exception('only integers allowed')
        if right < 2:
            raise Exception('power must be >= 2')
        grad_fn = PowNode(self, right)
        newData = self.data**right
        return Tensor(newData,grad_fn=grad_fn,requires_grad=True)


    # Backward computations. Will be implemented in Task 4.
    def backward(self, grad_output=None):
        # We first check if this tensor has a grad_fn: that is, one of the
        # nodes that you defined in Task 3.
        if self.grad_fn is not None:
            # If grad_fn is defined, we have computed this tensor using some operation.
            if grad_output is None:
                # This is the starting point of the backward computation.
                # This will typically be the tensor storing the output of
                # the loss function, on which we have called .backward()
                # in the training loop.
                grad_output = 1.0
                # This is an intermediate node in the computational graph.
                # This corresponds to any intermediate computation, such as
                # a hidden layer.
            self.grad_fn.backward(grad_output)
        else:
            # If grad_fn is not defined, this is an endpoint in the computational
            # graph: learnable model parameters or input data.
            if self.requires_grad:
                # This tensor *requires* a gradient to be computed. This will
                # typically be a tensor that holds learnable parameters.
                if self.grad is None:
                    self.grad = grad_output
                else:
                    self.grad += grad_output
            else:
                # This tensor *does not require* a gradient to be computed. This
                # will typically be a tensor holding input data.
                pass


# A small utility where we simply create a Tensor object. We use this to
# mimic torch.tensor.
def tensor(data, requires_grad=False):
    return Tensor(data, requires_grad)

# We define helper functions to implement the various arithmetic operations.

def tanh(t):

    out_data = np.tanh(t.data)
    requires_grad = t.requires_grad
    grad_fn = TanhNode(t)
    return Tensor(out_data, requires_grad=requires_grad, grad_fn=grad_fn)

def sigmoid(t):
    out_data = 1 / (1 + np.exp(-t.data))
    requires_grad = t.requires_grad
    grad_fn = SigmoidNode(t)
    return Tensor(out_data, requires_grad=requires_grad, grad_fn=grad_fn)

def bce_loss(pred, target):

    p = pred.data
    t = target.data
    loss_val = - (t * np.log(p + 1e-8) + (1 - t) * np.log(1 - p + 1e-8))
    loss_tensor = Tensor(np.array(loss_val), requires_grad=pred.requires_grad, grad_fn=BCE_LossNode(pred, target))
    return loss_tensor
# This function takes two tensors as input, and returns a new tensor holding
# the result of an element-wise addition on the two input tensors.
def addition(left, right):
    new_data = left.data + right.data
    grad_fn = AdditionNode(left, right) # TODO build a node in the computational graph (Task 3)
    return Tensor(new_data,grad_fn=grad_fn,requires_grad=True)

Some sanity checks.

In [14]:
# Two tensors holding row vectors.
x1 = tensor(np.array([[2.0, 3.0]]))
x2 = tensor(np.array([[1.0, 4.0]]))
# A tensors holding a column vector.
w = tensor(np.array([[-1.0], [1.2]]))

# Test the arithmetic operations.
test_plus = x1 + x2
test_minus = x1 - x2
test_power = x2 ** 2
test_matmul = x1 @ w

print(f'Test of addition: {x1.data} + {x2.data} = {test_plus.data}')
print(f'Test of subtraction: {x1.data} - {x2.data} = {test_minus.data}')
print(f'Test of power: {x2.data} ** 2 = {test_power.data}')
print(f'Test of matrix multiplication: {x1.data} @ {w.data} = {test_matmul.data}')

# Check that the results are as expected. Will crash if there is a miscalculation.
assert(np.allclose(test_plus.data, np.array([[3.0, 7.0]])))
assert(np.allclose(test_minus.data, np.array([[1.0, -1.0]])))
assert(np.allclose(test_power.data, np.array([[1.0, 16.0]])))
assert(np.allclose(test_matmul.data, np.array([[1.6]])))

Test of addition: [[2. 3.]] + [[1. 4.]] = [[3. 7.]]
Test of subtraction: [[2. 3.]] - [[1. 4.]] = [[ 1. -1.]]
Test of power: [[1. 4.]] ** 2 = [[ 1. 16.]]
Test of matrix multiplication: [[2. 3.]] @ [[-1. ]
 [ 1.2]] = [[1.6]]


# Tasks 3 and 4

In [13]:
class Node:
    def __init__(self):
        pass

    def backward(self, grad_output):
        raise NotImplementedError('Unimplemented')

    def __repr__(self):
        return str(type(self))


class AdditionNode(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def backward(self, grad_output):
        # TODO: in Task 4, implement backward step for the addition operation.
        if self.left.requires_grad:
            self.left.backward(grad_output)
        if self.right.requires_grad:
            self.right.backward(grad_output)

class SubNode(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def backward(self, grad_output):
        # TODO: in Task 4, implement backward step for the addition operation.
        if self.left.requires_grad:
            self.left.backward(grad_output)
        if self.right.requires_grad:
            self.right.backward(-grad_output)

class MatMulNode(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def backward(self, grad_output):
        # TODO: in Task 4, implement backward step for the addition operation.
        if self.left.requires_grad:
            self.left.backward(grad_output @ self.right.data.T)  # dL/dA = dL/dC @ B.T
        if self.right.requires_grad:
            self.right.backward(self.left.data.T @ grad_output)  # dL/dB = A.T @ dL/dC

class PowNode(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def backward(self, grad_output):
        if self.left.requires_grad:
            self.left.backward(self.right * (self.left.data ** (self.right - 1)) * grad_output)

class TanhNode(Node):
    def __init__(self, input_tensor):
        self.input_tensor = input_tensor
        self.out = np.tanh(input_tensor.data)

    def backward(self, grad_output):

        grad_input = grad_output * (1 - self.out**2)
        if self.input_tensor.requires_grad:
            self.input_tensor.backward(grad_input)



class SigmoidNode(Node):
    def __init__(self, input_tensor):
        self.input_tensor = input_tensor
        self.out = 1 / (1 + np.exp(-input_tensor.data))

    def backward(self, grad_output):

        grad_input = grad_output * self.out * (1 - self.out)
        if self.input_tensor.requires_grad:
            self.input_tensor.backward(grad_input)


class BCE_LossNode(Node):
    def __init__(self, pred, target):
        self.pred = pred
        self.target = target

    def backward(self, grad_output):
        p = self.pred.data
        t = self.target.data
        grad_pred = (-t / (p + 1e-8) + (1 - t) / (1 - p + 1e-8)) * grad_output
        if self.pred.requires_grad:
            self.pred.backward(grad_pred)

Sanity check for Task 3.

In [15]:
x = tensor(np.array([[2.0, 3.0]]))
w1 = tensor(np.array([[1.0, 4.0]]), requires_grad=True)
w2 = tensor(np.array([[3.0, -1.0]]), requires_grad=True)

test_graph = x + w1 + w2

print('Computational graph top node after x + w1 + w2:', test_graph.grad_fn)

assert(isinstance(test_graph.grad_fn, AdditionNode))
assert(test_graph.grad_fn.right is w2)
assert(test_graph.grad_fn.left.grad_fn.left is x)
assert(test_graph.grad_fn.left.grad_fn.right is w1)

Computational graph top node after x + w1 + w2: <class '__main__.AdditionNode'>


Sanity check for Task 4.

In [16]:
x = tensor(np.array([[2.0, 3.0]]))
w = tensor(np.array([[-1.0], [1.2]]), requires_grad=True)
y = tensor(np.array([[0.2]]))

# We could as well write simply loss = (x @ w - y)**2
# We break it down into steps here if you need to debug.

model_out = x @ w
diff = model_out - y
loss = diff ** 2

loss.backward()

print('Gradient of loss w.r.t. w =\n', w.grad)

assert(np.allclose(w.grad, np.array([[5.6], [8.4]])))
assert(x.grad is None)
assert(y.grad is None)

Gradient of loss w.r.t. w =
 [[5.6]
 [8.4]]


An equivalent cell using PyTorch code. Your implementation should give the same result for `w.grad`.

In [17]:
pt_x = torch.tensor(np.array([[2.0, 3.0]]))
pt_w = torch.tensor(np.array([[-1.0], [1.2]]), requires_grad=True)
pt_y = torch.tensor(np.array([[0.2]]))

pt_model_out = pt_x @ pt_w
pt_model_out.retain_grad() # Keep the gradient of intermediate nodes for debugging.

pt_diff = pt_model_out - pt_y
pt_diff.retain_grad()

pt_loss = pt_diff ** 2
pt_loss.retain_grad()

pt_loss.backward()
pt_w.grad

tensor([[5.6000],
        [8.4000]], dtype=torch.float64)

# Task 5

In [18]:
class Optimizer:
    def __init__(self, params):
        self.params = params

    def zero_grad(self):
        for p in self.params:
            p.grad = np.zeros_like(p.data)

    def step(self):
        raise NotImplementedError('Unimplemented')


class SGD(Optimizer):
    def __init__(self, params, lr):
        super().__init__(params)
        self.lr = lr

    def step(self):
        for p in self.params:
            if p.grad is not None:
                p.data -= self.lr * p.grad

In [19]:
np.random.seed(1)

w_init = np.random.normal(size=(2, 1))
b_init = np.random.normal(size=(1, 1))

# We just declare the parameter tensors. Do not use nn.Linear.
w = tensor(w_init,requires_grad=True)# TODO: a tensor initialized as w_init
b = tensor(b_init,requires_grad=True)# TODO: a tensor initialized as b_init

eta = 1e-2
opt = SGD([w, b], lr=eta)# TODO: a SGD optimizer with a learning rate of eta

for i in range(10):

    sum_err = 0

    for row in range(X.shape[0]):
        x = tensor(X[[row], :])
        y = tensor(Y[[row]])

        # Forward pass.
        y_pred = x @ w + b# TODO: compute predicted value for x
        err = (y_pred - y) ** 2   # TODO: compute squared error loss

        # Backward and update.
        # TODO: compute gradients and then update the model.
        opt.zero_grad()
        err.backward()
        opt.step()
        # For statistics.
        sum_err += err.item()

    mse = sum_err / X.shape[0]
    print(f'Epoch {i+1}: MSE =', mse)

Epoch 1: MSE = 0.7999661130823178
Epoch 2: MSE = 0.017392390107906875
Epoch 3: MSE = 0.009377418010839892
Epoch 4: MSE = 0.009355326971438456
Epoch 5: MSE = 0.009365440968904256
Epoch 6: MSE = 0.009366989180952533
Epoch 7: MSE = 0.009367207398577986
Epoch 8: MSE = 0.009367238983974489
Epoch 9: MSE = 0.009367243704122532
Epoch 10: MSE = 0.009367244427185763


# Task 6

**For Colab users:** the following command downloads the second CSV file.

In [20]:
!wget --no-check-certificate https://www.cse.chalmers.se/~richajo/dit866/assignments/a4/data/raisins.csv

--2025-02-28 11:07:59--  https://www.cse.chalmers.se/~richajo/dit866/assignments/a4/data/raisins.csv
Resolving www.cse.chalmers.se (www.cse.chalmers.se)... 129.16.222.93
Connecting to www.cse.chalmers.se (www.cse.chalmers.se)|129.16.222.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68230 (67K) [text/csv]
Saving to: ‘raisins.csv’

raisins.csv         100%[===================>]  66.63K   256KB/s    in 0.3s    

2025-02-28 11:08:00 (256 KB/s) - ‘raisins.csv’ saved [68230/68230]



In [21]:
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

a4data = pd.read_csv('raisins.csv')

X = scale(a4data.drop(columns='Class'))
Y = 1.0*(a4data.Class == 'Besni').to_numpy()

np.random.seed(0)
shuffle = np.random.permutation(len(Y))
X = X[shuffle]
Y = Y[shuffle]

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, random_state=0, test_size=0.2)
Ytrain = Ytrain.reshape(-1, 1)
Ytest = Ytest.reshape(-1, 1)

In [ ]:
Xtrain.shape, Ytrain.shape

In [22]:
d_in = Xtrain.shape[1]
d_hidden = 4
W1 = tensor(np.random.randn(d_in, d_hidden), requires_grad=True)
b1 = tensor(np.zeros((1, d_hidden)), requires_grad=True)
W2 = tensor(np.random.randn(d_hidden, 1), requires_grad=True)
b2 = tensor(np.zeros((1, 1)), requires_grad=True)

def model(x):
    z1 = x @ W1 + b1
    a1 = tanh(z1)

    z2 = a1 @ W2 + b2
    out = sigmoid(z2)
    return out
learning_rate = 0.01
epochs = 100
N = Xtrain.shape[0]

for epoch in range(epochs):
    epoch_loss = 0.0

    for i in range(N):
        x_i = tensor(Xtrain[i:i+1, :])
        y_i = tensor(Ytrain[i:i+1, :])
        pred = model(x_i)
        loss = bce_loss(pred, y_i)
        epoch_loss += loss.data.item()
        loss.backward()

        for param in [W1, b1, W2, b2]:
            param.data = param.data - learning_rate * param.grad
            param.grad = None
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {epoch_loss / N:.4f}")


correct = 0
N_test = Xtest.shape[0]
for i in range(N_test):
    x_i = tensor(Xtest[i:i+1, :])
    pred = model(x_i)
    pred_label = 1 if pred.data.item() > 0.5 else 0
    if pred_label == int(Ytest[i][0]):
        correct += 1
accuracy = correct / N_test
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

Epoch 10, Loss: 0.3429
Epoch 20, Loss: 0.3327
Epoch 30, Loss: 0.3254
Epoch 40, Loss: 0.3191
Epoch 50, Loss: 0.3139
Epoch 60, Loss: 0.3092
Epoch 70, Loss: 0.3055
Epoch 80, Loss: 0.3032
Epoch 90, Loss: 0.3017
Epoch 100, Loss: 0.3007
Test Accuracy: 85.56%
